In [1]:
from elasticsearch import Elasticsearch
import pandas as pd
from transformers import AutoTokenizer
import re
import math
from tqdm import tqdm

In [2]:
df = pd.read_json('../data/wikipedia_documents.json').T

In [3]:
df.head()

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4


In [4]:
df = df.drop_duplicates(subset=['text'])
df.nunique()

text             56737
corpus_source        1
url                  1
domain               0
title            31726
author               0
html                 0
document_id      56737
dtype: int64

In [6]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

In [130]:
df['text'][60]

'KETEL의 초기화면\n\n\n\u3000CONNECT 240\n\u3000[[[[[[[\n\u3000### ketel에 연결되었습니다.(korea:pc\n\u3000### Welcome to ketel!!\n\u3000### 서비스 명에 ketel을 입력하십시오\n\u3000### Please type "ke" for KETEL English Service\n\u3000서비스 명(login):kete\n\n\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000KK\u3000 KKK\u3000\u3000EEEEE\u3000TTTTTTTT\u3000\u3000EEEEEE\u3000 Lt\n\u3000\u3000\u3000\u3000\u3000\u3000 KK\u3000KK\u3000\u3000\u3000EE\u3000\u3000\u3000\u3000 \u3000TT\u3000\u3000\u3000 EE\u3000\u3000\u3000\u3000\n\u3000\u3000\u3000\u3000\u3000\u3000KKK\u3000\u3000\u3000\u3000 EEEE\u3000\u3000\u3000\u3000TT\u3000\u3000\u3000EEEE\u3000\u3000 L\n\u3000\u3000\u3000\u3000\u3000 KK KK\u3000\u3000\u3000 EE\u3000\u3000\u3000\u3000\u3000TT\u3000\u3000\u3000EE\u3000\u3000\u3000\u3000L\n\u3000\u3000\u3000\u3000\u3000KK\u3000 KKK\u3000EEEEEE\u3000\u3000 TT\u3000\u3000\u3000EEEEEE\u3000 LLLLLLL\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000 <<<알립니다>>\n\u30001.<<KETEL 정보서비스 일시 정지 안내>\n\u30002.KETE

In [134]:
len(tokenizer.tokenize(df['text'][60]))

440

In [158]:
end_of_sentences = re.findall(r'[가-힣]+\S*\.', df['text'][60])

In [159]:
sentences = re.split(r'[가-힣]+\S\.', df['text'][60])

In [160]:
print(end_of_sentences, len(end_of_sentences))
print(sentences, len(sentences))

['연결되었습니다.', '마감...'] 2
['KETEL의 초기화면\n\n\n\u3000CONNECT 240\n\u3000[[[[[[[\n\u3000### ketel에 ', '(korea:pc\n\u3000### Welcome to ketel!!\n\u3000### 서비스 명에 ketel을 입력하십시오\n\u3000### Please type "ke" for KETEL English Service\n\u3000서비스 명(login):kete\n\n\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000KK\u3000 KKK\u3000\u3000EEEEE\u3000TTTTTTTT\u3000\u3000EEEEEE\u3000 Lt\n\u3000\u3000\u3000\u3000\u3000\u3000 KK\u3000KK\u3000\u3000\u3000EE\u3000\u3000\u3000\u3000 \u3000TT\u3000\u3000\u3000 EE\u3000\u3000\u3000\u3000\n\u3000\u3000\u3000\u3000\u3000\u3000KKK\u3000\u3000\u3000\u3000 EEEE\u3000\u3000\u3000\u3000TT\u3000\u3000\u3000EEEE\u3000\u3000 L\n\u3000\u3000\u3000\u3000\u3000 KK KK\u3000\u3000\u3000 EE\u3000\u3000\u3000\u3000\u3000TT\u3000\u3000\u3000EE\u3000\u3000\u3000\u3000L\n\u3000\u3000\u3000\u3000\u3000KK\u3000 KKK\u3000EEEEEE\u3000\u3000 TT\u3000\u3000\u3000EEEEEE\u3000 LLLLLLL\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000 <<<알립니다>>\n\u30001.<<KETEL 정보서비스 일시

In [ ]:
max_possible_length = 510
partial_text = ''
for i in range(len(sentences)-1):
  
  # print(len(tokenizer.tokenize(sentences[i] + end_of_sentences[i])))
  max_possible_length -= len(tokenizer.tokenize(sentences[i] + end_of_sentences[i]))
  if max_possible_length >= 0:
    partial_text += sentences[i] + end_of_sentences[i]
  else:
    

In [156]:
splited_documents = []

In [157]:
count = 0
for index, document in df.iterrows():
  print("==================count=====================",count)
  count +=1
  original_passage = document['text']

  length_of_tokenized_text = len(tokenizer.tokenize(original_passage))

  number_of_passages_to_create = math.ceil(length_of_tokenized_text / 400)  # 510으로 안하는 이유는 110만큼의 buffer를 주기 위해서
  min_required_length = length_of_tokenized_text // number_of_passages_to_create  # 최소 길이
  max_possible_length = min_required_length + 110  # 최대길이

  end_of_sentences = re.findall(r'[가-힣]+\S*\.', original_passage)
  sentences = re.split(r'[가-힣]+\S*\.', original_passage)
  sentence_counter = 0


  print(min_required_length)
  for i in range(number_of_passages_to_create):
    partial_text = ''
    partial_text_length = 0

    if sentence_counter == len(end_of_sentences)-1:
      break

    for j in range(len(sentences) - 1):
      partial_text += sentences[sentence_counter] + end_of_sentences[sentence_counter]
      partial_text_length += len(tokenizer.tokenize(sentences[sentence_counter] + end_of_sentences[sentence_counter]))
      print('==============================', sentence_counter, len(sentences)-1, len(end_of_sentences)-1)
      print(partial_text, partial_text_length)
      sentence_counter += 1
      if partial_text_length >= min_required_length or sentence_counter == len(end_of_sentences)-1:
        print('들어왔니?')
        if sentence_counter == len(end_of_sentences)-1:
          partial_text += sentences[-1]
        document_to_be_added = {
          'text': partial_text,
          'title': document['title'],
          'document_id': document['document_id']
        }

        splited_documents.append(document_to_be_added)
        break




  #   if length_of_tokenized_text < 510:
  #     document_to_be_added = {
  #       'text': document['text'],
  #       'title': document['title'],
  #       'document_id': document['document_id']
  #     }

  #     splited_documents.append(document_to_be_added)
    
  #   else:

  
  # count += 1
  # if count > 5:
  #   break

==================count===================== 0
223
============================== 0 8 7
이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다. 29
============================== 1 8 7
이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.

이 목록은 명료화를 위해 두 부분으로 나뉘어 있다. 44
============================== 2 8 7
이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.

이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.

# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다. 84
============================== 3 8 7
이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.

이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.

# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.
# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다. 128
============================== 4 8 7
이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.

이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.

# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았

IndexError: list index out of range

In [145]:
splited_documents

[{'text': 'KETEL의 초기화면\n\n\n\u3000CONNECT 240\n\u3000[[[[[[[\n\u3000### ketel에 연결되었습니다.(korea:pc\n\u3000### Welcome to ketel!!\n\u3000### 서비스 명에 ketel을 입력하십시오\n\u3000### Please type "ke" for KETEL English Service\n\u3000서비스 명(login):kete\n\n\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000KK\u3000 KKK\u3000\u3000EEEEE\u3000TTTTTTTT\u3000\u3000EEEEEE\u3000 Lt\n\u3000\u3000\u3000\u3000\u3000\u3000 KK\u3000KK\u3000\u3000\u3000EE\u3000\u3000\u3000\u3000 \u3000TT\u3000\u3000\u3000 EE\u3000\u3000\u3000\u3000\n\u3000\u3000\u3000\u3000\u3000\u3000KKK\u3000\u3000\u3000\u3000 EEEE\u3000\u3000\u3000\u3000TT\u3000\u3000\u3000EEEE\u3000\u3000 L\n\u3000\u3000\u3000\u3000\u3000 KK KK\u3000\u3000\u3000 EE\u3000\u3000\u3000\u3000\u3000TT\u3000\u3000\u3000EE\u3000\u3000\u3000\u3000L\n\u3000\u3000\u3000\u3000\u3000KK\u3000 KKK\u3000EEEEEE\u3000\u3000 TT\u3000\u3000\u3000EEEEEE\u3000 LLLLLLL\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000 <<<알립니다>>\n\u30001.<<KETEL 정보서비스 일시 정지 안내>\n\u

In [104]:
len(splited_documents)

3